# Importação das bibliotecas

In [29]:
import pandas as pd
import matplotlib.pyplot as plt 
from sqlalchemy import create_engine, inspect, text

# Dados da empresa

In [4]:
url_itens_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/itens_pedidos.csv'
url_pedidos = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/pedidos.csv'
url_produto = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/produtos.csv'
url_vendedores = 'https://github.com/alura-cursos/SQL-python-integracao/raw/main/TABELAS/vendedores.csv'


# Leitura dos dados da empresa

In [5]:
itens_pedidos = pd.read_csv(url_itens_pedidos) # type: ignore
pedidos = pd.read_csv(url_pedidos) # type: ignore
produtos = pd.read_csv(url_produto) # type: ignore
vendedores = pd.read_csv(url_vendedores) # type: ignore

# Criação do Banco de Dados

In [8]:
engine = create_engine("sqlite:///:memory:")

# Colocando os dados no Banco de Dados

In [9]:
produtos.to_sql('produtos', engine,index=False)
pedidos.to_sql('pedidos', engine,index=False)
vendedores.to_sql('vendedores', engine,index=False)
itens_pedidos.to_sql('itens_pedidos', engine,index=False)

24527

# Criação da função "consulta"

In [17]:
def consulta(query):
    with engine.connect() as conexao:
        consulta = conexao.execute(text(query))
        dados =  consulta.fetchall()
    return pd.DataFrame(dados,columns=consulta.keys())

# Filtragem e criação do gráfico para análise  

In [20]:
query= "SELECT CONDICAO, COUNT(*) AS 'QUANTIDADE' FROM PRODUTOS GROUP BY CONDICAO"
df_produtos = consulta(query)
plt.bar(df_produtos["Condicao"], df_produtos["QUANTIDADE"], color='#9353FF' )
plt.title("Quantidade de Produtos por Condicao")
plt.show()

In [1]:
# consulta("SELECT * FROM PRODUTOS").head(3)
# consulta("SELECT*FROM ITENS_PEDIDOS").head(3)

# Filtragem e criação do gráfico para análise  

In [53]:
query = '''SELECT PRODUTOS.PRODUTO, SUM(ITENS_PEDIDOS.QUANTIDADE) AS QUANTIDADE
FROM ITENS_PEDIDOS,PRODUTOS 
WHERE ITENS_PEDIDOS.PRODUTO_ID = PRODUTOS.PRODUTO_ID 
GROUP BY PRODUTOS.PRODUTO
ORDER BY QUANTIDADE ASC'''

df_prod_quant =consulta(query)
plt.barh(df_prod_quant["produto"][-10:], df_prod_quant["QUANTIDADE"][-10:])
plt.xlabel('quantidadevemdida')
plt.show()

# Filtragem para fazer a análise

In [67]:
query = '''SELECT VENDEDORES.NOME_VENDEDOR, AVG(PEDIDOS.TOTAL) AS 'VALOR'
FROM PEDIDOS, VENDEDORES
WHERE strftime('%Y', data_compra)= '2020' AND VENDEDORES.VENDEDOR_ID = PEDIDOS.VENDEDOR_ID
GROUP BY VENDEDORES.NOME_VENDEDOR
ORDER BY AVG(PEDIDOS.TOTAL) DESC
'''
consulta(query)

,nome_vendedor,VALOR
0,Daniel Siqueira,1972.338526
1,Millena Pereira,1894.614343
2,Ana Duarte,1843.452582
3,Nadia Oliveira,1818.342200
4,Paulo Calanca,1813.469855


# Filtragem para fazer a análise

In [69]:
query ='''SELECT ESTADO, COUNT(*) AS PEDIDOS
FROM ITENS_PEDIDOS
GROUP BY ESTADO
ORDER BY PEDIDOS DESC
'''
consulta(query)

,Estado,PEDIDOS
0,BR-MA,974
1,BR-DF,953
2,BR-MT,937
3,BR-GO,933
4,BR-PA,932
5,BR-AL,928
6,BR-PE,927
7,BR-RR,925
8,BR-RO,925
9,BR-RN,921


# Filtragem para fazer a análise

In [81]:
query= '''SELECT VENDEDORES.NOME_VENDEDOR, COUNT(*) AS QUANTIDADE_VENDAS
FROM PEDIDOS
JOIN VENDEDORES ON VENDEDORES.VENDEDOR_ID= PEDIDOS.VENDEDOR_ID
JOIN ITENS_PEDIDOS ON ITENS_PEDIDOS.PEDIDO_ID = PEDIDOS.PEDIDO_ID
WHERE ITENS_PEDIDOS.ESTADO = 'BR-SP'
GROUP BY VENDEDORES.NOME_VENDEDOR
ORDER BY QUANTIDADE_VENDAS DESC
'''
consulta(query)

,nome_vendedor,QUANTIDADE_VENDAS
0,Daniel Siqueira,190
1,Paulo Calanca,182
2,Nadia Oliveira,181
3,Ana Duarte,180
4,Millena Pereira,157
